In [1]:
import ee
import geopandas
import numpy as np
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import folium
from shapely.geometry import box
import warnings
from functools import reduce
from math import sin, cos, sqrt, atan2, radians

Create a dictionary of countries with its boundary coordinates

In [25]:
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
country_polygon_dict = dict()
for index, row in world.iterrows():
    if 'MultiPolygon' in str(type(row['geometry'])):
        i = 0
        for poly in row['geometry']:
            # Remove small area that's not needed for sampling
            if poly.area > 5:
                # Area is in the unit of squre degrees
                country_polygon_dict[row['iso_a3']+'_'+str(i)] = (poly, round(poly.area*0.2))
                i +=1 
    # Here implies a Polygon
    elif row['geometry'].area > 5:
        country_polygon_dict[row['iso_a3']] = (row['geometry'], round(row['geometry'].area*0.2))

In [26]:
# Remove miscellaneous countries
del country_polygon_dict['-99']
del country_polygon_dict['-99_0']
del country_polygon_dict['-99_1']
del country_polygon_dict['-99_2']

In [27]:
country_polygon_dict

{'AFG': (<shapely.geometry.polygon.Polygon at 0x7efe5bfcd828>, 13),
 'AGO_0': (<shapely.geometry.polygon.Polygon at 0x7efe5c0b59e8>, 21),
 'ARE': (<shapely.geometry.polygon.Polygon at 0x7efe5c0b1390>, 1),
 'ARG_0': (<shapely.geometry.polygon.Polygon at 0x7efe5c181cf8>, 55),
 'ATA_0': (<shapely.geometry.polygon.Polygon at 0x7efe5c0bc240>, 4),
 'ATA_1': (<shapely.geometry.polygon.Polygon at 0x7efe5c0bca20>, 3),
 'ATA_2': (<shapely.geometry.polygon.Polygon at 0x7efe5c0bc9b0>, 1197),
 'AUS_0': (<shapely.geometry.polygon.Polygon at 0x7efe5c0adf98>, 1),
 'AUS_1': (<shapely.geometry.polygon.Polygon at 0x7efe5c0ad438>, 138),
 'AUT': (<shapely.geometry.polygon.Polygon at 0x7efe5bfd1ac8>, 2),
 'AZE_0': (<shapely.geometry.polygon.Polygon at 0x7efe5c0bc3c8>, 2),
 'BEN': (<shapely.geometry.polygon.Polygon at 0x7efe5bfc1da0>, 2),
 'BFA': (<shapely.geometry.polygon.Polygon at 0x7efe5c185438>, 5),
 'BGD': (<shapely.geometry.polygon.Polygon at 0x7efe5bfccc18>, 2),
 'BGR': (<shapely.geometry.polygon.Pol

Pull Sentinel-2 Data

In [2]:
def satellite_imagery_Sentinel2(center_lat, center_lon, edge_len,
                      start_date, end_date, 
                      plot_option, resolution=30):
    
    area_of_interest_ee = ee.Geometry.Rectangle([center_lon-edge_len/2, center_lat-edge_len/2, center_lon+edge_len/2, center_lat+edge_len/2])
    
    area_of_interest_shapely = box(center_lon-edge_len/2, center_lat-edge_len/2, center_lon+edge_len/2, center_lat+edge_len/2)
    
    def calc_distance(lon1, lat1, lon2, lat2):
        # Reference: https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude
        # approximate radius of earth in km
        R = 6373.0
        lon1 = radians(lon1)
        lat1 = radians(lat1)
        lon2 = radians(lon2)
        lat2 = radians(lat2)
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        distance = R * c
        return distance
    
    print('The selected area is approximately {:.2f} km by {:.2f} km'.format(\
                calc_distance(center_lon-edge_len/2, center_lat, center_lon+edge_len/2, center_lat), \
                calc_distance(center_lon, center_lat-edge_len/2, center_lon, center_lat+edge_len/2)))
    
    # Create image collection that contains the area of interest
    img_collect = (ee.ImageCollection('COPERNICUS/S2')
                 .filterDate(start_date, end_date)
                 .filterBounds(area_of_interest_ee)
                    # Remove image that's too small (likely to be partial image)
                    # Size of a full image: 1,276,131,371; size of a partial image: 276,598,191
                 .filter(ee.Filter.gt('system:asset_size', 1200000000))
    
#         img_collect = img_collect.filterMetadata("CLOUDY_PIXEL_PERCENTAGE","less_than",20) 
    
    assert (img_collect.size().getInfo()>0), "No valid image"
    print("Total number of images in the collection: ", img_collect.size().getInfo())
        
    # Extract tile information from each image
    # Note: tiles can overlap a little bit
    unique_tiles = set([item['properties']['MGRS_TILE'] for item in img_collect.getInfo()['features']])
    if len(unique_tiles) > 1:
        warnings.warn('Multiple tiles are selected. Proceed with caution.')
        print('Number of tiles selected: ', len(unique_tiles))
    if img_collect_no_partial.size().getInfo() < img_collect.size().getInfo():
        warnings.warn('There are partial images in the collection. Proceed with caution.')
        print('Number of partial images: ', img_collect.size().getInfo()-img_collect_no_partial.size().getInfo())
        
    if 'Sentinel' in source:
        # Reference: https://www.satimagingcorp.com/satellite-sensors/other-satellite-sensors/sentinel-2a/
        band_blue = 'B2' #10m
        band_green = 'B3' #10m
        band_red = "B4"  #10m
        band_nir = 'B8'  #10m
    # Reference: https://landsat.gsfc.nasa.gov/landsat-data-continuity-mission/
    elif 'Landsat7' in source:
        # Reference: https://www.usgs.gov/land-resources/nli/landsat/landsat-7?qt-science_support_page_related_con=0#qt-science_support_page_related_con
        band_blue = 'B1' #30m
        band_green = 'B2' #30m
        band_red = "B3"  #30m
        band_nir = 'B4'  #30m
    elif 'Landsat8' in source:
        # Reference: https://www.usgs.gov/faqs/what-are-best-landsat-spectral-bands-use-my-research?qt-news_science_products=0#qt-news_science_products
        band_blue = 'B2' #30m
        band_green = 'B3' #30m
        band_red = "B4"  #30m
        band_nir = 'B5'  #30m
    
    def calc_NDVI(img):
        ndvi = ee.Image(img.normalizedDifference([band_nir, band_red])).rename(["ndvi"]).copyProperties(img, img.propertyNames())
        composite = img.addBands(ndvi)
        return composite
    
    # SAVI = ((NIR – Red) / (NIR + Red + L)) x (1 + L)
    def calc_SAVI(img):
        """A function to compute Soil Adjusted Vegetation Index."""
        savi =  ee.Image(img.expression(
            '(1 + L) * float(nir - red)/ (nir + red + L)',
            {
                'nir': img.select(band_nir),
                'red': img.select(band_red),
                'L': 0.5
            })).rename(["savi"]).copyProperties(img, img.propertyNames())
        composite = img.addBands(savi)
        return composite

    # EVI = 2.5 * ((NIR – Red) / ((NIR) + (C1 * Red) – (C2 * Blue) + L))
    #     C1=6, C2=7.5, and L=1
    def calc_EVI(img):
        """A function to compute Soil Adjusted Vegetation Index."""
        evi = ee.Image(img.expression(
          '(2.5) * float(nir - red)/ ((nir) + (C1*red) - (C2*blue) + L)',
          {   
              'nir': img.select(band_nir),
              'red': img.select(band_red),
              'blue': img.select(band_blue),
              'L': 0.2,
              'C1': 6,
              'C2': 7.5
          })).rename(["evi"]).copyProperties(img, img.propertyNames())
        composite = img.addBands(evi)
        return composite
                   
    def add_landcover(img):
        landcover = gfsad
        composite = img.addBands(landcover)
        return composite
    
    def calc_YYYYMM(img):
        return img.set('YYYYMM', img.date().format("YYYYMM"))
    
    def add_ee_layer(self, ee_object, vis_params, name):
        try:    
            if isinstance(ee_object, ee.image.Image):    
                map_id_dict = ee.Image(ee_object).getMapId(vis_params)
                folium.raster_layers.TileLayer(
                    tiles = map_id_dict['tile_fetcher'].url_format,
                    attr = 'Google Earth Engine',
                    name = name,
                    overlay = True,
                    control = True
                    ).add_to(self)
            elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
                ee_object_new = ee_object.median()
                map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
                folium.raster_layers.TileLayer(
                    tiles = map_id_dict['tile_fetcher'].url_format,
                    attr = 'Google Earth Engine',
                    name = name,
                    overlay = True,
                    control = True
                    ).add_to(self)
            elif isinstance(ee_object, ee.geometry.Geometry):    
                folium.GeoJson(
                        data = ee_object.getInfo(),
                        name = name,
                        overlay = True,
                        control = True
                    ).add_to(self)
            elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
                ee_object_new = ee.Image().paint(ee_object, 0, 2)
                map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
                folium.raster_layers.TileLayer(
                        tiles = map_id_dict['tile_fetcher'].url_format,
                        attr = 'Google Earth Engine',
                        name = name,
                        overlay = True,
                        control = True
                    ).add_to(self)

        except:
            print("Could not display {}".format(name))

    # Add EE drawing method to folium.
    folium.Map.add_ee_layer = add_ee_layer
    
    img_collect_calc = img_collect.map(calc_YYYYMM).map(calc_NDVI).map(calc_SAVI).map(calc_EVI).map(add_landcover)
    
    unique_month = list(set([item['properties']['YYYYMM'] for item in img_collect_calc.getInfo()['features']]))
    unique_month.sort()
    
    if len(unique_month) > 0:
        warnings.warn('There are null values in the output DataFrame. Proceed with caution.')
    
    img_calc_month_dict = dict()
    temp_dict = dict()
    for month in unique_month:
        img_calc_month_dict[month] = img_collect_calc.filter(ee.Filter.eq('YYYYMM',month)).median()
        img_calc_month2 = img_calc_month_dict[month].addBands(ee.Image.pixelLonLat())
        data_month_lst = img_calc_month2.reduceRegion(reducer=ee.Reducer.toList(), \
                                                             geometry=area_of_interest_ee, maxPixels=1e13, scale=resolution)
        lat_series = pd.Series(np.array((ee.Array(data_month_lst.get("latitude")).getInfo())), name="lat")
        lon_series = pd.Series(np.array((ee.Array(data_month_lst.get("longitude")).getInfo())), name="lon")
        ndvi_series = pd.Series(np.array((ee.Array(data_month_lst.get("ndvi")).getInfo())), name=month+'_NDVI')
        savi_series = pd.Series(np.array((ee.Array(data_month_lst.get("savi")).getInfo())), name=month+'_SAVI')
        evi_series = pd.Series(np.array((ee.Array(data_month_lst.get("evi")).getInfo())), name=month+'_EVI')
        temp_dict[month] = pd.concat([lat_series, lon_series, ndvi_series, savi_series, evi_series], axis=1)
    
    df_lst = list(temp_dict.values())
    out_df = reduce(lambda left, right: pd.merge(left,right,on=['lat', 'lon']), df_lst)

    # Output the column names that have null values
    if len(out_df.columns[out_df.isnull().any()]) > 0:
        warnings.warn('There are null values in the output DataFrame. Proceed with caution.')

    # Create a folium map object.
    myMap = folium.Map(location=[center_lat, center_lon], zoom_start=8)
    # Add the box around the area of interest
    folium.GeoJson(area_of_interest_shapely, name="Area of Interest").add_to(myMap)

    visParams = {'min':0, 'max':1, 'palette': ['red', 'yellow', 'green']}
    if plot_option == 'NDVI':
        for month in unique_month:
            myMap.add_ee_layer(img_calc_month_dict[month].select("ndvi"), visParams, name=source+' '+plot_option+' '+month)
    elif plot_option == 'SAVI':
        for month in unique_month:
            myMap.add_ee_layer(img_calc_month_dict[month].select("savi"), visParams, name=source+' '+plot_option+' '+month)
    elif plot_option == 'EVI':
        for month in unique_month:
            myMap.add_ee_layer(img_calc_month_dict[month].select("evi"), visParams, name=source+' '+plot_option+' '+month)

    # Add a layer control panel to the map.
    myMap.add_child(folium.LayerControl())

    return myMap, out_df